In [2]:
# Declarations
import glob
import json
import os
import numpy as np
import pandas as pd

from os.path import basename

In [3]:
# DataFrame name json file
nameFile = []
pathFile = []
rootFile = []

path = os.getcwd()
for root, dirs, files in os.walk('../../../SLIDE-x-D001-Artix-7-Recipe/SLIDE-x-RES/RECIPE_INT'):
    for file in files:
        if file.endswith(".json"):
            nameFile.append(file)
            pathFile.append(os.path.join(root, file))
            rootFile.append(root)
            
nameFileS = pd.Series(nameFile)
pathFileS = pd.Series(pathFile)
rootFileS = pd.Series(rootFile)

pd.options.display.max_colwidth = 100
print('nameFileS:'+nameFileS)
print('pathFileS:'+pathFileS)
print('rootFileS:'+rootFileS)

dfNF = pd.concat([nameFileS, pathFileS, rootFileS], axis=1)
dfNF.columns = ['FILE_NAME', 'FILE_PATH', 'FILE_ROOT']

0     nameFileS:parameters.json
1     nameFileS:parameters.json
2     nameFileS:parameters.json
3     nameFileS:parameters.json
4     nameFileS:parameters.json
5     nameFileS:parameters.json
6     nameFileS:parameters.json
7     nameFileS:parameters.json
8     nameFileS:parameters.json
9     nameFileS:parameters.json
10    nameFileS:parameters.json
11    nameFileS:parameters.json
12    nameFileS:parameters.json
13    nameFileS:parameters.json
dtype: object
0             pathFileS:../../../SLIDE-x-D001-Artix-7-Recipe/SLIDE-x-RES/RECIPE_INT/prime/parameters.json
1     pathFileS:../../../SLIDE-x-D001-Artix-7-Recipe/SLIDE-x-RES/RECIPE_INT/insertionsort/parameters.json
2       pathFileS:../../../SLIDE-x-D001-Artix-7-Recipe/SLIDE-x-RES/RECIPE_INT/park_miller/parameters.json
3          pathFileS:../../../SLIDE-x-D001-Artix-7-Recipe/SLIDE-x-RES/RECIPE_INT/bsort100/parameters.json
4                pathFileS:../../../SLIDE-x-D001-Artix-7-Recipe/SLIDE-x-RES/RECIPE_INT/bs/parameters.json
5       

In [4]:
# .json file loop
df = pd.DataFrame(columns=['DEVICE','FUNCTION','DATA_TYPE',
                           'NUM_S','RANGE_S','NUM_SI','RANGE_SI',
                           'NUM_ARR','RANGE_ARR'])  # 'BOARD',

# listD = ["Arm","Atmega328p","Leon3","RiscV","Thumb"] # Inserisci i dispositivi che vuoi analizzare
listD = ["Bambu"]
listDT = ["int8_t","int16_t","int32_t","int64_t"] # Inserisci i data type che vuoi analizzare

In [5]:
def analyze_variables(strings):
    unique_strings = set()  # to keep track of unique strings without brackets 
    used_in_others = set()  # to keep track of strings used within brackets in other strings (Indexes)
    two_brackets = 0  # to count strings with exactly one pair of brackets (Arrays)
    four_brackets = 0  # to count strings with exactly two pairs of brackets (Matrixes)

    for string in strings:
        if '[' not in string and ']' not in string:
            # This is a unique string without brackets
            unique_strings.add(string)
        else:
            # This string contains brackets, find substrings inside brackets
            inside_brackets = string.split('[')[1:]
            inside_brackets = [s.split(']')[0] for s in inside_brackets]
            used_in_others.update(inside_brackets)

            # Count the number of brackets
            open_brackets = string.count('[')
            close_brackets = string.count(']')
            
            if open_brackets == 1 and close_brackets == 1:
                two_brackets += 1
            elif open_brackets == 2 and close_brackets == 2:
                four_brackets += 1

    # Find strings that appear only once and do not contain brackets
    unique_no_brackets = [s for s in unique_strings if strings.count(s) == 1 and s not in used_in_others] # (scalar value)

    return {
        'Scalars': len(unique_no_brackets),
        'Indexes': len(used_in_others),
        'Arrays': two_brackets,
        'Matrixes': four_brackets
    }

for i in range(0, len(pathFile)):
    pF = pathFile[i]
    print('pF:'+str(pF))

    with open(pF, 'r') as jf:
        data = jf.read()
    obj = json.loads(data)

    if os.name == 'nt':
        spR = rootFile[i].split('\\')
        print('spR:'+str(spR))
    else:
        spR = rootFile[i].split('/')
        print('spR:'+str(spR))
    
    function = spR[-1]
    print('function:'+str(function))
    # board = listD[i]
    # print('board:'+str(board))
    
    #Device loop
    deviceList = list(obj.keys())
    print('deviceList:'+str(deviceList))
    deviceListR = []
    for u in range(0, len(deviceList)):
        if deviceList[u] in listD:
            deviceListR.append(deviceList[u])
            print('deviceListR:'+str(deviceList[u]))

    for n in range(0, len(deviceListR)):
        # Data_type loop
        device = deviceListR[n]
        print('device:'+str(device))
        dtarr = list(obj[deviceListR[n]].keys())
        print('dtarr:'+str(dtarr))
        
        dtarrR = []
        for u in range(0, len(dtarr)):
            if dtarr[u] in listDT:
                dtarrR.append(dtarr[u])
                print(f'dtarr[{u}]:'+str(dtarr[u]))
        
        for j in range(0, len(dtarrR)):
            dataype = dtarrR[j]
            print('dataype:'+str(dtarrR[j]))
            lsubNP = (list(obj[deviceListR[n]][dtarrR[j]].keys()))
            print('lsubNP:'+str(lsubNP))
            lsubVP = (list(obj[deviceListR[n]][dtarrR[j]].values()))
            print('lsubVP:'+str(lsubVP))
            
            # sublist dimension
            ltot = len(lsubNP)
            print('ltot:'+str(ltot))
        
            # how many array are present
            na = 0
            nam = 0
            fl = 0
            flm = 0
            
            ark = 0
            arkm = 0
            
            for k in range(0, ltot): 
                #print(function, fl)
                if "[" in lsubNP[k] and (not "][" in lsubNP[k]):
                    na = na + 1
                    if fl == 0:
                        ark = k
                        fl = 1
                elif "[" in lsubNP[k] and "][" in lsubNP[k]:
                    nam = nam + 1
                    if flm == 0:
                        arkm = k
                        flm = 1

            '''
            # how many scalar index are present
            nsi = 0
            for k in range(0, ltot):
                flag = 0
                if not "[" in lsubNP[k]:
                    for keysl in lsubNP:
                        if (lsubNP[k] in keysl) and (not lsubNP[k] is keysl) and (flag == 0):          
                            flag = 1
                            nsi = nsi + 1           

            # how many scalar are present
            ns = ltot - na - nsi
            '''

            # check if is it negative
            ranArr = 0
            if na != 0:
                newstr = lsubVP[ark].replace("[", "")
                newstr = newstr.replace("]", "")
                newstr = newstr.split(",")[0]
                num = float(newstr)
                if num < 0:
                    ranArr = 1
                else:
                    ranArr = 0
                
            # check if is it negative
            ranMat = 0
            if nam != 0:
                newstrM = lsubVP[arkm].replace("[", "")
                newstrM = newstrM.replace("]", "")
                newstrM = newstrM.split(",")[0]
                numM = float(newstr)
                if numM < 0:
                    ranMat = 1
                else:
                    ranMat = 0

            '''
            df = df.append({'DEVICE': device, 'FUNCTION': function, 'DATA_TYPE': dataype,
                            'NUM_S': ns, 'RANGE_S': 0,'NUM_SI': nsi,'RANGE_SI': 0,
                            'NUM_ARR': na,'RANGE_ARR': ranArr}, 
                            ignore_index=True)  # 'BOARD': board, 
            '''
            
            result = analyze_variables(lsubNP)
                
            df = df.append({'DEVICE': device, 'FUNCTION': function, 'DATA_TYPE': dataype,
                'NUM_S': result['Scalars'], 'RANGE_S': 0,'NUM_SI': result['Indexes'],'RANGE_SI': 0,
                'NUM_ARR': result['Arrays'],'RANGE_ARR': ranArr, 'NUM_MAT': result['Matrixes'],'RANGE_MAT': ranMat}, 
                ignore_index=True)  # 'BOARD': board, 

pF:../../../SLIDE-x-D001-Artix-7-Recipe/SLIDE-x-RES/RECIPE_INT/prime/parameters.json
spR:['../../../SLIDE-x-D001-Artix-7-Recipe/SLIDE-x-RES/RECIPE_INT', 'prime']
function:prime
deviceList:['args', 'ALL', 'Arm', 'Thumb', '8051', 'Atmega328p', 'Leon3', 'RiscV', 'Bambu']
deviceListR:Bambu
device:Bambu
dtarr:['int8_t', 'uint8_t', 'int16_t', 'int32_t', 'int64_t']
dtarr[0]:int8_t
dtarr[2]:int16_t
dtarr[3]:int32_t
dtarr[4]:int64_t
dataype:int8_t
lsubNP:['n']
lsubVP:['[2,127];100']
ltot:1
dataype:int16_t
lsubNP:['n']
lsubVP:['[2,32766];100']
ltot:1
dataype:int32_t
lsubNP:['n']
lsubVP:['[2,2147483640];100']
ltot:1
dataype:int64_t
lsubNP:['n']
lsubVP:['[2,2147483640];100']
ltot:1
pF:../../../SLIDE-x-D001-Artix-7-Recipe/SLIDE-x-RES/RECIPE_INT/insertionsort/parameters.json
spR:['../../../SLIDE-x-D001-Artix-7-Recipe/SLIDE-x-RES/RECIPE_INT', 'insertionsort']
function:insertionsort
deviceList:['args', 'ALL', 'Arm', 'Thumb', 'Bambu', '8051', 'Atmega328p', 'Leon3', 'RiscV']
deviceListR:Bambu
device:Bam

In [6]:
dfS = pd.read_csv("dfS.csv", sep=';')
dfSI = pd.read_csv("dfSI.csv", sep=';')

In [7]:
dfS = pd.read_csv("dfS_tot.csv", sep=';')
dfSI = pd.read_csv("dfSI_tot.csv", sep=';')

In [8]:
del dfS['Unnamed: 0']
del dfSI['Unnamed: 0']

In [9]:
columnsS = ['DEVICE','FUNCTION','DATA_TYPE','RANGE_S']
columnsSI = ['DEVICE','FUNCTION','DATA_TYPE','RANGE_SI']
dfS = dfS[columnsS]
dfSI = dfSI[columnsSI]

In [10]:
lent = len(dfS['RANGE_S'].tolist())
for i in range(0, lent):
    df.loc[(df['DEVICE'] == dfS['DEVICE'][i]) & 
           # (df['BOARD'] == dfS['BOARD'][i]) &
           (df['FUNCTION'] == dfS['FUNCTION'][i]) & 
           (df['DATA_TYPE'] == dfS['DATA_TYPE'][i]), 'RANGE_S'] = dfS['RANGE_S'][i]

In [11]:
lent = len(dfSI['RANGE_SI'].tolist())
for i in range(0, lent):
    df.loc[(df['DEVICE'] == dfSI['DEVICE'][i]) & 
           # (df['BOARD'] == dfSI['BOARD'][i]) & 
           (df['FUNCTION'] == dfSI['FUNCTION'][i]) & 
           (df['DATA_TYPE'] == dfSI['DATA_TYPE'][i]), 'RANGE_SI'] = dfSI['RANGE_SI'][i]

In [12]:
df.to_csv("RangeValueClassification_tot.csv", sep=';')